In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor
import matplotlib.pyplot as plt
import graphviz
%matplotlib inline

# Problem 1

In [2]:
df_train = pd.read_csv("./Concrete_train.csv")

In [3]:
df_test = pd.read_csv("./Concrete_test.csv")

## (A)

In [4]:
features_train = df_train.drop(df_train.columns[-1], axis=1)
features_test = df_test.drop(df_test.columns[-1],axis=1)
features_train.head()

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day)
0,233.81,0.0,94.58,197.89,4.567,947.04,852.16,28
1,251.81,0.0,99.94,146.14,12.350,1006.00,899.76,56
2,252.00,0.0,0.00,186.00,0.000,1111.00,784.00,7
3,325.60,166.4,0.00,174.00,8.900,881.60,790.00,28
4,250.00,0.0,95.69,191.84,5.330,948.90,857.20,14


In [5]:
features_test.head()

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day)
0,236.00,0.0,0.00,193.00,0.00,968.00,885.0,7
1,288.00,121.0,0.00,177.00,7.00,908.00,829.0,28
2,290.35,0.0,96.18,168.08,9.41,961.18,865.0,14
3,302.00,0.0,0.00,203.00,0.00,974.00,817.0,28
4,262.00,111.0,86.00,195.00,5.00,895.00,733.0,28


In [6]:
target_train = df_train.iloc[:,-1]
target_test = df_test.iloc[:,-1]

In [10]:
grid = {'n_estimators':[100,500,2500],'max_depth': np.arange(1,30)}
rtree = RandomForestRegressor(n_jobs=-1)

In [11]:
rtreeCV = GridSearchCV(rtree,param_grid=grid,return_train_score=True)
rtreeCV.fit(features_train,target_train)
print()
print('optimal depth= ', rtreeCV.best_params_, 'validation R2= ', rtreeCV.best_score_.round(3))


optimal depth=  {'max_depth': 23, 'n_estimators': 100} validation R2=  0.884


## (B)

In [26]:
grid = {'learning_rate':[0.1,0.01,0.001],'n_estimators':[100,500,2500],'max_depth':np.arange(1,30)}
gbt = GradientBoostingRegressor()
gbtCV = GridSearchCV(gbt,param_grid=grid,n_jobs=-1)
gbtCV.fit(features_train,target_train)
print('optimal depth= ', rtreeCV.best_params_, 'validation R2= ', rtreeCV.best_score_.round(3))

C:\Users\rileyma\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


optimal depth=  {'max_depth': 23, 'n_estimators': 100} validation R2=  0.884


## (C)

In [28]:
rf_opt = rtreeCV.best_estimator_
R2_train = rf_opt.score(features_train,target_train)
R2_test  = rf_opt.score(features_test,target_test)
print('FORESTS    train R2 =',R2_train.round(3),'  test R2 =',R2_test.round(3))
gbt = gbtCV.best_estimator_
R2_train = gbt.score(features_train,target_train)
R2_test  = gbt.score(features_test,target_test)
print('GBT        train R2 =',R2_train.round(3),'  test R2 =',R2_test.round(3))

FORESTS    train R2 = 0.982   test R2 = 0.889
GBT        train R2 = 0.995   test R2 = 0.906


- The GBT is more accurate, shown by the better train and test R2 scores
- The Forests takes longer, as it doesn't have a learning rate to speed things up
- GBT, as it does the learning for us